In [119]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import time
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [120]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [121]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [122]:
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)

Mars Now


In [104]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)
response = requests.get(url)
soup = bs(browser.html, 'html.parser')

In [105]:
featured_image_url = soup.find('img', class_='headerimage fade-in')
featured_image_url = url + featured_image_url['src']
#done with pics

In [106]:
url = "https://galaxyfacts-mars.com/"
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [107]:
table = pd.read_html(url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [108]:
df = table[0]
df.columns = ['Stat', 'Mars', 'delete']
del df['delete']
df = df.drop(labels=0, axis=0)
df

,Stat,Mars
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days
6,Temperature:,-87 to -5 °C


In [109]:
df2= table[1]
df2.columns=['Stat', 'Mars']
df2

,Stat,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [110]:
dfs = [df, df2]
dfs = pd.concat(dfs,ignore_index=True)
dfs


,Stat,Mars
0,Diameter:,"6,779 km"
1,Mass:,6.39 × 10^23 kg
2,Moons:,2
3,Distance from Sun:,"227,943,824 km"
4,Length of Year:,687 Earth days
5,Temperature:,-87 to -5 °C
6,Equatorial Diameter:,"6,792 km"
7,Polar Diameter:,"6,752 km"
8,Mass:,6.39 × 10^23 kg (0.11 Earths)
9,Moons:,2 ( Phobos & Deimos )


In [111]:
facts=dfs.to_dict("records")
#donewithfacts


In [112]:
url = 'https://marshemispheres.com'
browser.visit(url)
response = requests.get(url)
soup = bs(response.text, "html.parser")

In [113]:
soup = soup.find_all('div', class_='item')

In [114]:
hem_urls=[]

for x in soup:
    
    #using soup grab title 
    title= x.h3.text
    #find  page with image
    links= x.find("a", class_="itemLink product-item")['href']
    browser.visit(f"https://marshemispheres.com/{links}")
    #find the link
    img_url = browser.find_by_text('Sample')['href']
    #append to dict
    hem_urls.append({'title': title, 'img_url': img_url})

print (hem_urls)




[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [115]:
mars_data = {
        "titles": titles,
        "paragraphs": paragraphs,
        "featured_image_url": featured_image_url,
        "facts": facts,
        "hem_urls": hem_urls
        }